### How to use this script:
This script can be used for two purposes:
1. Determine optimal size for scenes to be cropped (2nd cell executed with CALC_CROP_SIZE = True)
2. Generate crops of scenes (3rd cell with predetermined crop size or calculated crop size on the fly). 
3. Generate crops of flows
----------
There are two options (indicated by the flag PADDING): 
- Padded crops (region of interest changing size and the remaining space filled with zeros)(recommended for scenes?)
- Non-padded crops (corresponds a "cropped frame" with the camera panning and following the acter as he moves along) (recommended for flows)

In [1]:
print("Settings.py should be e.g. crops")
from gait_analysis import CasiaDataset
from gait_analysis.Config import Config
import numpy as np

from gait_analysis import CasiaDataset
from gait_analysis.Config import Config
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from PIL import Image

c = Config()
c.config['indexing']['grouping'] = 'person_sequence_angle'

# CROP_SIZE = [156.0, 230.0] # for five angles and five persons

MARGIN = 10
CROP_SIZE = [156.0, 230.0] # maybe default crop size
CALC_CROP_SIZE = False
CROP_FLOWS = True
if not CALC_CROP_SIZE:
    IMAGE_OUTPUT_SIZE = [CROP_SIZE[0]+2*MARGIN,CROP_SIZE[1]+2*MARGIN]
    print("image output size",IMAGE_OUTPUT_SIZE)
else:
    print("image output size to be determined with [CROP_SIZE[0]+2*MARGIN,CROP_SIZE[1]+2*MARGIN]")
PADDING = False


Settings.py should be e.g. crops
loading configuration  crops
[OK]
image output size [176.0, 250.0]


In [2]:
dataset = CasiaDataset()

if CALC_CROP_SIZE:
    print("Starting calculation of crop sizes")
    x_max_list = []
    y_max_list = []
    invalid_pose_counter = 0
    print("Using",dataset[0]['poses'].shape[0],"poses")
    for item in dataset: 
    #for i in range(5):
        #for listitem in dataset: print(listitem)
        annotations = item['annotations']
        scenes = item['scenes']
        poses = item['poses']
    #     print("Nr scenes, poses:", len(scenes),poses.shape)
        #print("Currently in", dataset.dataset_items[i])
        #print("Shapes:")
        #print(poses.shape)
        x_tmp = []
        y_tmp = []
        for scene,nr in zip(scenes,range(poses.shape[2])):
            pose = poses[:,:,nr]
            p_x = pose[:,0]
            p_y = pose[:,1]
    #         print(p_x)
    #         print(p_y)
            if (all(p_x == 0) or all(p_y == 0)):
                print("Problematic pose in ")
            else:
                x_max = np.max([p for p in p_x if p != 0])
                x_min = np.min([p for p in p_x if p != 0])
                abs_delta_x = x_max-x_min
                x_max_list.append(abs_delta_x)
                y_max = np.max([p for p in p_y if p != 0])
                y_min = np.min([p for p in p_y if p != 0])
                abs_delta_y = y_max-y_min
                y_max_list.append(abs_delta_y) 
#                 if (abs_delta_x > 640):
#                     print("Wrong pose",pose)

    CROP_SIZE = [np.ceil(max(x_max_list)), np.ceil(max(y_max_list))]
    # print("Crop size as double:",max(x_max_list),max(y_max_list))
    print("Crop size without margin:",CROP_SIZE)
    IMAGE_OUTPUT_SIZE = [CROP_SIZE[0]+2*MARGIN,CROP_SIZE[1]+2*MARGIN]
print("image output size",IMAGE_OUTPUT_SIZE)

image output size [176.0, 250.0]


In [3]:
# print("Using", poses.shape[0],"poses")
invalid_pose_counter = 0
last_person = '000'

for item, i in zip(dataset, range(len(dataset))):  
    annotations = item['annotations']
    
    if CROP_FLOWS:
        data_in = item['flows']
    else:
        data_in = item['scenes']
    
    poses = item['poses']
    
    #since valid scenes have an offset respect to thier annotation number
    annotations_offset = int(annotations[''][0])
    
    person = '{:03d}'.format(dataset.dataset_items[i][0]) 
    sequence = dataset.dataset_items[i][1]
    angle = '{:03d}'.format(dataset.dataset_items[i][2])
    origin = '/mnt/DATA/HIWI/IBT/CASIA/'
    folderpath_for_debug_only = origin + 'images/'+ person + '/'+ sequence + '/' + angle + '/'
    if CROP_FLOWS:
        pathlib.Path(origin + 'preprocessing/crops_flow/'+ person + '/' + sequence + '/' + sequence + '-' + angle + '/'
).mkdir(parents=True, exist_ok = True)
    else:
        pathlib.Path(origin + 'preprocessing/crops/'+ person + '/' + sequence + '/' + sequence + '-' + angle + '/'
).mkdir(parents=True, exist_ok = True)
#     print("current folder: ", folderpath)
    
    if last_person != person:
        print("Person", person)
    last_person = person

    x0_last = 1
    x1_last = 1
    y0_last = 1 + CROP_SIZE[0]
    y1_last = 1 + CROP_SIZE[1]
    
    for j in range(len(data_in)):
        pose_x = poses[:,0,j]
        pose_y = poses[:,1,j]
#         print(pose_x)
        if (all(x == 0 for x in pose_x) or all(y == 0 for y in pose_y)):
            print("WARNING: Invalid poses in ", folderpath_for_debug_only, "pose nr",j, "taking last available pose (or crop size if first)")
            x0 = x0_last
            x1 = x1_last
            y0 = y0_last
            y1 = y1_last
            invalid_pose_counter += 1
        else:
            # HERE x is horizontal direction and y is vertical direction
            x0 = np.floor(np.min(poses[np.nonzero(pose_x),0,j])) - MARGIN
            x1 = np.floor(np.max(poses[np.nonzero(pose_x),0,j])) + MARGIN
            y0 = np.floor(np.min(poses[np.nonzero(pose_y),1,j])) - MARGIN
            y1 = np.floor(np.max(poses[np.nonzero(pose_y),1,j])) + MARGIN
            
        #make sure image borders stay in range
        x0 = max(0,x0)
        x1 = min(x1,data_in[j].shape[1])
        y0 = max(0,y0)
        y1 = min(y1,data_in[j].shape[0])

#       padding image until it gets the desired size
        x_to_pad = IMAGE_OUTPUT_SIZE[0] - (x1 - x0)
        y_to_pad = IMAGE_OUTPUT_SIZE[1] - (y1 - y0)
        x_pad_l = np.floor(x_to_pad/2)
        x_pad_r = x_to_pad - x_pad_l
        y_pad_l = np.floor(y_to_pad/2)
        y_pad_r = y_to_pad - y_pad_l
        
        if PADDING:
            im_tmp = data_in[j][int(y0):int(y1),int(x0):int(x1)]
            im_final = np.pad(im_tmp,[(int(y_pad_l),int(y_pad_r)),(int(x_pad_l),int(x_pad_r)),(0,0)],'constant')#,'constant', constant_values=((0, 0),(0,0)))
        else: #NOT PADDING IMAGE WITH ZEROS
            x0 = x0-x_pad_l
            x1 = x1+x_pad_r
            y0 = y0-y_pad_l
            y1 = y1+y_pad_r
#             print("Coordinates before",x0_n,x1_n,y0_n,y1_n)
            if x0 < 0:
                x1 = x1 - x0
                x0 = 0
            elif data_in[j].shape[1] < x1:
                x0 = x0 - x1 + data_in[j].shape[1]
                x1 = data_in[j].shape[1]
            if y0 < 0:
                y1 = y1 - y0
                y1 = 0
            elif data_in[j].shape[0] < y1:
                y0 = y0 - y1 + data_in[j].shape[0]
                y1 = data_in[j].shape[0]            
            
            im_final = data_in[j][int(y0):int(y1),int(x0):int(x1)]

            if (x1-x0 != IMAGE_OUTPUT_SIZE[0] or y1-y0 != IMAGE_OUTPUT_SIZE[1]):
                print("WRONG IMAGE COORDINATES DETECTED ", folderpath_for_debug_only, "pose nr",j)
                print("Coordinates after",x0,x1,y0,y1)
        plt.imshow(im_final)
        framename = '{:03d}'.format(j+annotations_offset)
        if CROP_FLOWS:
            saving_path = origin + 'preprocessing/crops_flow/'+ person + '/' + sequence + '/' + sequence + '-' + angle + '/'    
            filename = person + '-' + sequence + '-' + angle + '_frame_' + framename + '_flow.png'
        else:
            saving_path = origin + 'preprocessing/crops/'+ person + '/' + sequence + '/' + sequence + '-' + angle + '/'
            filename = person + '-' + sequence + '-' + angle + '_frame_' + framename + '_crop.png'
        totalpath = saving_path + filename
#         print("Image name", filename)
#         print("Total path", totalpath)
        Image.fromarray(im_final).save(totalpath)
        x0_last = x0
        x1_last = x1
        y0_last = y0
        y1_last = y1

print("Done")

Person 001
[376.392 378.991 380.338 398.568 393.389 394.662 389.445 395.964 394.651
 378.965 371.159 381.669]
[373.781 378.967 380.354 393.409 390.743 390.726 377.714 382.948 392.075
 371.145 369.828 382.959]
[372.472 376.405 380.35  393.341 389.395 385.523 371.154 380.259 389.452
 369.901 368.57  384.228]
[371.214 373.772 379.009 392.072 384.227 380.333 369.86  377.676 381.621
 371.141 368.558 381.605]
[371.121 372.476 380.279 390.775 381.663 376.372 367.274 373.8   375.092
 376.399 371.176 385.52 ]
[369.823 371.238 381.59  390.696 380.348 369.919 367.225 371.204 369.841
 380.265 373.786 386.828]
[367.253 371.204 381.652 388.117 379.002 368.541 362.031 369.834 368.523
 379.017 376.375 388.134]
[363.33  369.873 381.597 382.949 376.376 368.547 362.049 369.862 367.261
 377.711 373.81  388.126]
[360.773 363.362 378.983 381.62  373.776 367.274 360.771 369.855 365.946
 376.395 371.186 381.66 ]
[359.455 360.687 373.8   381.555 372.456 365.938 360.677 368.539 364.663
 369.839 364.654 380.338]